In [30]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/Colab\ Notebooks/kaggle
    from setup_colab import setup_colab_for_kaggle
    setup_colab_for_kaggle(check_env=False, local_working=True)
except:
    print("Not in Colab")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/kaggle
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Content of Drive Kaggle data dir (/content/drive/MyDrive/kaggle): ['/content/drive/MyDrive/kaggle/input', '/content/drive/MyDrive/kaggle/working', '/content/drive/MyDrive/kaggle/.ipynb_checkpoints', '/content/drive/MyDrive/kaggle/output']
Content of Kaggle data dir (/kaggle): ['/kaggle/input', '/kaggle/output', '/kaggle/working']
Content of Kaggle data subdir (/kaggle/input): ['/kaggle/input/cassava-model', '/kaggle/input/cassava-leaf-disease-classification', '/kaggle/input/googlebitemperedloss', '/kaggle/input/vbdyolo', '/kaggle/input/.ipynb_checkpoints', '/kaggle/input/vinbigdata', '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection', '/kaggle/input/vinbigdata-chest-xray-origi

In [31]:
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

INPUT_FOLDER = Path("/kaggle/input/vinbigdata-chest-xray-abnormalities-detection")
INPUT_FOLDER_ORIGINAL_PNG = Path("/kaggle/input/vinbigdata-chest-xray-original-png")
INPUT_FOLDER_YOLO_OUT = Path("/kaggle/output/vbdyolo_out")
WORK_FOLDER = Path("/kaggle/working")

In [32]:
def read_prediction_labels(filename: Path, image_w: int, image_h: int):
    if not filename.exists():
        return "14 1 0 0 1 1"

    labels: pd.DataFrame = pd.read_csv(filename, delimiter=" ", header=None)
    labels.columns = ["class_id", "x_centre", "y_centre", "bw", "bh", "conf"]

    # Convert YOLO format (x_centre, y_centre, bw, bh) to competition format (x_min, y_min, x_max, y_max)
    labels["x_min"] = labels["x_centre"] - labels["bw"] / 2
    labels["y_min"] = labels["y_centre"] - labels["bh"] / 2
    labels["x_max"] = labels["x_centre"] + labels["bw"] / 2
    labels["y_max"] = labels["y_centre"] + labels["bh"] / 2
    labels = labels.drop(columns=["x_centre", "y_centre", "bw", "bh"])
    # After dropping, conf column should become the second one.
    assert(labels.columns.to_list() == ["class_id", "conf", "x_min", "y_min", "x_max", "y_max"])

    # Scale coordinates to image's size. Clip to make sure it's not out of bounds of the image.
    labels[["x_min", "x_max"]] = (labels[["x_min", "x_max"]] * image_w).round().astype(np.int32).clip(0, image_w - 1)
    labels[["y_min", "y_max"]] = (labels[["y_min", "y_max"]] * image_h).round().astype(np.int32).clip(0, image_h - 1)

    # Convert all rows to one prediction string
    return " ".join(labels.to_string(header=False, index=False).split())

In [33]:
results_df = pd.DataFrame(columns=["image_id", "PredictionString"])

test_metadata = pd.read_csv(INPUT_FOLDER_ORIGINAL_PNG / "test_meta.csv")
test_metadata = test_metadata.set_index("image_id").to_dict("index")

for image_id, image_dims in tqdm(test_metadata.items(), total=len(test_metadata)):
    prediction_str = read_prediction_labels(
        INPUT_FOLDER_YOLO_OUT / "labels_pred" / f"{image_id}.txt", image_dims["dim0"], image_dims["dim1"]
    )
    results_df = results_df.append({"image_id": image_id, "PredictionString": prediction_str}, ignore_index=True)

results_df.to_csv(WORK_FOLDER / "submission.csv", index=False)
display(results_df.sample(10))

,image_id,PredictionString
1947,a9653a3944debde919a631ede6f9ac5e,3 0.84668 679 1308 1631 1620
177,0ed15ab15f2304a9a811d7fa6ce843de,3 0.856934 693 1511 1481 1838
2949,fc853c552cdae3eb0eb78f6101922b06,5 0.571289 210 1091 952 2024 0 0.642090 1128 7...
2156,b933b11d1a87912401b3839c0624bf48,14 1 0 0 1 1
139,0c84dad44979c3a7777a8f4f4d9f5f7a,14 1 0 0 1 1
2445,d154ed5ece3a6da5cadf6658a7076cc7,3 0.803223 894 1236 1641 1564 0 0.836914 1147 ...
1194,6b94fef0e417c083388ba961b1d2786a,3 0.806641 737 1305 1710 1674 0 0.839355 990 6...
2211,bd6d55f7df34a4347f106831229d0d18,14 1 0 0 1 1
2526,d922cc7124ae504b4e53b8d016c25cca,0 0.605469 1202 598 1518 981
243,165f2b132c2942833ab8138d48c96800,14 1 0 0 1 1


In [34]:
!mkdir -p /kaggle/output/vbdsubmit
!cp {WORK_FOLDER}/submission.csv /kaggle/output/vbdsubmit